# EDA on MSFT historical data
This notebook will use MSFT stock on the EDA.  
We will train the best model identified in the aapl notebook to test how well the model works on other stocks.

### Imports

In [16]:
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score


sys.path.append(str(Path("..").resolve()))
from src.ml_model.data import fetch_data, stock_data_feature_engineering
from src.ml_model.training import train_model, sequence_split

### Reading Data

In [17]:
df = fetch_data("MSFT", (2020, 1, 1), (2024, 12, 31))
df.head()

open     high       low   close  \
symbol timestamp                                                      
MSFT   2020-01-02 05:00:00+00:00  158.78  160.730  158.3300  160.62   
       2020-01-03 05:00:00+00:00  158.32  159.945  158.0600  158.62   
       2020-01-06 05:00:00+00:00  157.08  159.100  156.5100  159.03   
       2020-01-07 05:00:00+00:00  159.32  159.670  157.3200  157.58   
       2020-01-08 05:00:00+00:00  158.93  160.800  157.9491  160.09   

                                      volume  trade_count        vwap  
symbol timestamp                                                       
MSFT   2020-01-02 05:00:00+00:00  25472631.0     175506.0  159.815510  
       2020-01-03 05:00:00+00:00  24389239.0     166336.0  159.101120  
       2020-01-06 05:00:00+00:00  24709110.0     148395.0  158.495004  
       2020-01-07 05:00:00+00:00  24503429.0     167838.0  158.298227  
       2020-01-08 05:00:00+00:00  31748417.0     198632.0  159.714784

### Preprocessing and Visualization

In [ ]:
df_changed = df.copy()
X, y = stock_data_feature_engineering(df_changed)

\\wsl.localhost\Ubuntu\home\milad\repositories\Live-Trading-Platform\src\ml_model\data.py:139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = (df['close'].shift(-1) > df['close']).astype(int)


AttributeError: 'DataFrame' object has no attribute 'DataFrame'

### Training & Evaluation

In [ ]:
X_train, X_test, y_train, y_test = sequence_split(X, y)

model = train_model(X_train, y_train)

# Evaluate LSTM
y_pred = (model.predict(X_test) > 0.5).astype(int)
print("LSTM F1-score:", f1_score(y_test, y_pred))

NameError: name 'sequence_split' is not defined